In [1]:
import math
from itertools import combinations

In [20]:
def L(x, y, p=2):
    # x1 = [1, 1], x2 = [5,1]
    if len(x) == len(y) and len(x) > 1:
        sum = 0
        for i in range(len(x)):
            sum += math.pow(abs(x[i] - y[i]), p)
        return math.pow(sum, 1 / p)
    else:
        return 0

In [21]:
x1 = [1, 1]
x2 = [5, 1]
x3 = [4, 4]

In [27]:
for i in range(1,5):
    r = {'1-{}'.format(c): L(x1,c,p=i) for c in [x2,x3]}
    print(min(zip(r.values(),r.keys())))

(4.0, '1-[5, 1]')
(4.0, '1-[5, 1]')
(3.7797631496846193, '1-[4, 4]')
(3.5676213450081633, '1-[4, 4]')


python实现，遍历所有数据点，找出n个距离最近的点的分类情况，少数服从多数